### Use `execution_id` in tokenator to track usage across a set of executions

Most complex LLM workflows and AI agents use multiple LLM calls, possibly multiple LLM API providers, to achieve a task. With tokenator's `execution_id` field, one can easily track and monitor token usage and cost of an AI agent completing a task.

In this example, we are going to be analyzing a W2 tax form and understanding how much tax this person has to pay for the year 2024. Using, tokenator, we will run all kinds of analyses to build the most cost effective AI agent.

Let's start with installing some dependencies. I am going to be using the openai and anthropic models to complete this task.

In [1]:
%pip install e /Users/ujjwal/xfic/token-usage/ openai anthropic --quiet


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
You should consider upgrading via the '/Users/ujjwal/.pyenv/versions/3.10.0/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Next, we will setup 2 functions to access our model. This is done so that we don't have to initialize our openai and anthropic clients repeatedly. Also, we will be using structured outputs 

In [17]:
# Add API Keys here
OPENAI_API_KEY="openai-key"
ANTHROPIC_API_KEY="anthropic-key"

In [11]:
from tokenator import tokenator_openai, tokenator_anthropic

from openai import OpenAI
from anthropic import Anthropic

def call_openai_structured(system_prompt, user_prompt, image_url, model="gpt-4o", execution_id=None, response_format=None):
    client = tokenator_openai(OpenAI(api_key=OPENAI_API_KEY))

    messages = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": user_prompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_url,
                    },
                },
            ],
        }
    ]

    response = client.beta.chat.completions.parse(
        model=model,
        messages=messages,
        response_format=response_format,
        execution_id=execution_id,
    )

    return response

def call_openai_unstructured(system_prompt, user_prompt, model="gpt-4o", execution_id=None):
    client = tokenator_openai(OpenAI(api_key=OPENAI_API_KEY))

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": system_prompt},
                  {"role": "user", "content": user_prompt}],
        execution_id=execution_id,
    )

    return response

def call_anthropic(system_prompt, user_prompt, model="claude-3-5-sonnet-20241022", execution_id=None):
    client = tokenator_anthropic(Anthropic(api_key=ANTHROPIC_API_KEY))

    response = client.messages.create(
        model=model,
        system=system_prompt,
        messages=[{"role": "user", "content": user_prompt}],
        execution_id=execution_id,
        max_tokens=5000,
    )

    return response


Let's build our AI agent.
Here are the steps : 
- Get a W2 form image
- Pass it to OpenAI gpt 4o and get some income and tax related details
- Use these details and ask a model whether this person should be getting a tax refund or not

In [14]:
from pydantic import BaseModel

def execute_agent(image_url, execution_id, task1_model="gpt-4o", task2_provider="openai", task2_model="gpt-4o"):   

    # pydantic class for structured output
    class W2Details(BaseModel):
        name: str
        taxation_state: str
        income: float
        federal_tax_paid: float
        state_tax_paid: float
        social_security_tax_paid: float
        medicare_tax_paid: float

    user_prompt = f"Extract the following details from the W2 form image : {image_url}"

    system_prompt = "You are an expert at extracting details from W2 forms. You are given an image of a W2 form and you need to extract the fields required in the output"

    # 1st LLM call
    response = call_openai_structured(
        system_prompt=system_prompt, 
        user_prompt=user_prompt, 
        image_url=image_url, 
        model=task1_model, 
        response_format=W2Details, 
        execution_id=execution_id
    )

    print("Extracted details from W2 form : ")
    print(response.choices[0].message.content)


    # ------------------------------------------------------------------------------------------------

    # Now, we will use the extracted details to ask a model whether this person should be getting a tax refund or not
    user_prompt = f"Based on the following details, determine if this person should be getting a tax refund or not : {response.choices[0].message.content}"

    system_prompt = "You are an expert at determining whether a person should be getting a tax refund or not. You are given a set of details and you need to determine if this person should be getting a tax refund or not. Assume no deductions or credits are applied to the income"

    if task2_provider == "openai":
        response = call_openai_unstructured(
            system_prompt, 
            user_prompt, 
            model=task2_model,
            execution_id=execution_id
        )
        print("Refund status : ")
        print(response.choices[0].message.content)

    elif task2_provider == "anthropic":
        response = call_anthropic(
            system_prompt, 
            user_prompt, 
            model=task2_model,
            execution_id=execution_id
        )
        print("Refund status : ")
        print(response.content[0].text)




Our AI agent is all set!

The extra parameters such as task1_model, task2_provider, task2_model are hepful in experimenting with different models. 

Let's run the agent!

In [5]:
# get image
image_url = "https://www.patriotsoftware.com/wp-content/uploads/2024/03/2024-Form-W-2-1.png"


Let's try with gpt-4o model for all the LLM calls

In [9]:
# execute agent
import uuid

tokenator_execution_id = str(uuid.uuid4())
print(tokenator_execution_id)

execute_agent(image_url, tokenator_execution_id, task1_model="gpt-4o", task2_provider="openai", task2_model="gpt-4o")

6860675a-f506-4b48-8699-9c3c3f57f375
Extracted details from W2 form : 
{"name":"Abby L Smith","taxation_state":"OH","income":50000,"federal_tax_paid":4092,"state_tax_paid":1040.88,"social_security_tax_paid":3100,"medicare_tax_paid":725}
Refund status : 
To determine if Abby L Smith should be getting a tax refund, we need to calculate her correctly owed taxes based on her given details and compare them with the taxes she has already paid.

1. **Federal Taxes:**
   - For 2023, the federal income tax brackets for a single filer are:
     - 10% on income up to $11,000
     - 12% on income from $11,001 to $44,725
     - 22% on income from $44,726 to $95,375
   - Calculate her federal tax liability:
     - Tax on first $11,000 = $1,100 (10%)
     - Tax on income between $11,001 and $44,725 = ($44,725 - $11,000) * 12% = $4,041
     - Tax on income between $44,726 and $50,000 = ($50,000 - $44,725) * 22% = $1,159.50
     - Total federal tax owed = $1,100 + $4,041 + $1,159.50 = $6,300.50
   - Fe

Results seem interesting. But let's try out Anthropics claude Sonnet 3.5 to check if we can get a better explanation.

In [15]:
# execute agent
import uuid

tokenator_execution_id = str(uuid.uuid4())
print(tokenator_execution_id)

execute_agent(image_url, tokenator_execution_id, task1_model="gpt-4o", task2_provider="anthropic", task2_model="claude-3-5-sonnet-20241022")

d3c539ba-bc71-46a7-ad6b-88483c3fd294
Extracted details from W2 form : 
{"name":"Abby L Smith","taxation_state":"OH","income":50000,"federal_tax_paid":4092,"state_tax_paid":1040.88,"social_security_tax_paid":3100,"medicare_tax_paid":725}
Refund status : 
I'll help analyze if Abby L Smith should receive a tax refund. Let's break this down:

1. Federal Tax:
- Income: $50,000
- For 2023 tax brackets, on $50,000:
  * Standard federal tax liability would be approximately $5,553
  * Actual federal tax paid: $4,092
  * UNDERPAID by about $1,461

2. State Tax (Ohio):
- Ohio state tax on $50,000 would be approximately $1,030
- Actually paid: $1,040.88
- OVERPAID by about $10.88

3. Social Security Tax:
- Required rate is 6.2% of income
- On $50,000 should be: $3,100
- Actually paid: $3,100
- CORRECT amount paid

4. Medicare Tax:
- Required rate is 1.45% of income
- On $50,000 should be: $725
- Actually paid: $725
- CORRECT amount paid

Conclusion: NO, Abby should NOT expect a tax refund. She has

Hmmm interesting, claude sonnet seems to think the taxes are significantly underpaid.

You know what, let's throw a mountain of compute on this problem. Let's use openai o1.

In [18]:
# execute agent
import uuid

tokenator_execution_id = str(uuid.uuid4())
print(tokenator_execution_id)

execute_agent(image_url, tokenator_execution_id, task1_model="gpt-4o", task2_provider="openai", task2_model="o1")

99d9cfd9-d7ca-4057-9784-5acd1bff26af
Extracted details from W2 form : 
{"name":"Abby L Smith","taxation_state":"OH","income":50000.0,"federal_tax_paid":4092.0,"state_tax_paid":1040.88,"social_security_tax_paid":3100.0,"medicare_tax_paid":725.0}
Refund status : 
Below is a very basic breakdown (assuming 2023 rates and no deductions/credits at all) to illustrate why Abby would not be getting a refund.

────────────────────────────────────
1) FEDERAL TAX (SINGLE FILER, 2023)
────────────────────────────────────
• First $11,000 at 10%  → $1,100  
• Next $33,725 at 12%   → $4,047  (that is $44,725 – $11,000 = $33,725 × 12%)  
• Remaining $5,275 at 22% → $1,160.50 (that is $50,000 – $44,725 = $5,275 × 22%)  

Approx. Federal Tax Liability = $1,100 + $4,047 + $1,160.50 ≈ $6,307.50  

Federal Tax Paid = $4,092.00  
→ She is underpaid by about $6,307.50 – $4,092.00 = $2,215.50  

────────────────────────────────────────────
2) OHIO STATE TAX (2023 BRACKETS, NO CREDITS)
─────────────────────────

Awesome! `o1`, with it's extra compute, agrees with claude sonnet and thinks that Abby will have to pay additional taxes.
Now let's look at the cost of each execution.

In [22]:
from tokenator import usage

print("Usage with gpt-4o for both tasks : ")
print(usage.for_execution("6860675a-f506-4b48-8699-9c3c3f57f375").model_dump_json(indent=2))

print("------------------------------------------------------------------------------------------------")

print("\n\nUsage with gpt-4o for task 1 and claude sonnet for task 2 : ")
print(usage.for_execution("d3c539ba-bc71-46a7-ad6b-88483c3fd294").model_dump_json(indent=2))

print("------------------------------------------------------------------------------------------------")

print("\n\nUsage with gpt-4o for task 1 and o1 for task 2 : ")
print(usage.for_execution("99d9cfd9-d7ca-4057-9784-5acd1bff26af").model_dump_json(indent=2))



Usage with gpt-4o for both tasks : 
{
  "total_cost": 0.009503,
  "total_tokens": 1860,
  "prompt_tokens": 1213,
  "completion_tokens": 647,
  "prompt_tokens_details": null,
  "completion_tokens_details": null,
  "providers": [
    {
      "total_cost": 0.009503,
      "total_tokens": 1860,
      "prompt_tokens": 1213,
      "completion_tokens": 647,
      "prompt_tokens_details": null,
      "completion_tokens_details": null,
      "provider": "openai",
      "models": [
        {
          "total_cost": 0.009503,
          "total_tokens": 1860,
          "prompt_tokens": 1213,
          "completion_tokens": 647,
          "prompt_tokens_details": null,
          "completion_tokens_details": null,
          "model": "gpt-4o-2024-08-06"
        }
      ]
    }
  ]
}
------------------------------------------------------------------------------------------------


Usage with gpt-4o for task 1 and claude sonnet for task 2 : 
{
  "total_cost": 0.008446,
  "total_tokens": 1594,
  "prompt_t

With the help of tokenator, we are clearly able to determine what the token usage and cost related to each model is. We can see that even though openai `o1` gave us maybe the clearest formatted answer, it is clearly way more expensive than claude sonnet 3.5 and gpt-4o. Depending on your specific task, you should use evaluations to determine the accuracy of each model on your task. Then, using tokenator's cost analysis, choose the correct model for your task.